In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.data import AUTOTUNE
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np

NUM_CLASSES = 12
BATCH_SIZE = 32

In [2]:
# Model is trained on Google Colab T4 GPU
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/img_test.zip -d img_test

In [ ]:
!unzip /content/drive/MyDrive/img_valid.zip -d img_valid

In [ ]:
!unzip /content/drive/MyDrive/img_train.zip -d img_train

In [7]:
def load_dataset(csv_path, image_folder):
    df = pd.read_csv(csv_path)
    df['image_path'] = df['id'].apply(lambda x: os.path.join(image_folder, f"{x}.jpg"))
    df = df[df['image_path'].apply(os.path.exists)].reset_index(drop=True)
    df['multi_hot'] = df['annotations'].apply(lambda x: process_annotations(x, NUM_CLASSES))
    return df

In [10]:
def process_annotations(ann_str, num_classes=NUM_CLASSES):
    labels = [int(i) for i in ann_str.split(',')]
    unique_labels = sorted(set(labels), key=labels.index)
    multi_hot = np.zeros(num_classes, dtype=np.float32)
    for label in unique_labels:
        multi_hot[label] = 1
    return multi_hot

In [14]:
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, label

In [11]:
train_csv_path = "img_train.csv"
train_image_folder = "img_train/img_train/"
train_df = load_dataset(train_csv_path, train_image_folder)

In [12]:
train_image_paths = train_df['image_path'].values
train_labels = np.stack(train_df['multi_hot'].values)
train_ds = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels))

In [15]:
train_ds = train_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [16]:
val_csv_path = "img_valid.csv"
val_image_folder = "img_valid/img_valid/"
val_df = load_dataset(val_csv_path, val_image_folder)
val_image_paths = val_df['image_path'].values
val_labels = np.stack(val_df['multi_hot'].values)
val_ds = tf.data.Dataset.from_tensor_slices((val_image_paths, val_labels))
val_ds = val_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [17]:
test_csv_path = "img_test.csv"
test_image_folder = "img_test/img_test/"
test_df = load_dataset(test_csv_path, test_image_folder)
test_image_paths = test_df['image_path'].values
test_labels = np.stack(test_df['multi_hot'].values)
test_ds = tf.data.Dataset.from_tensor_slices((test_image_paths, test_labels))
test_ds = test_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [18]:
def compute_class_weights(train_labels):
    pos_counts = np.sum(train_labels, axis=0)
    neg_counts = len(train_labels) - pos_counts
    class_weights = neg_counts / (pos_counts + 1e-6)  # avoid division by 0
    return class_weights

In [19]:
def weighted_binary_crossentropy(class_weights):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        weights = tf.constant(class_weights, dtype=tf.float32)
        bce = - (y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
        weighted_bce = bce * weights
        return tf.reduce_mean(weighted_bce)
    return loss

In [20]:
def build_simple_cnn(input_shape=(224, 224, 3), num_classes=NUM_CLASSES):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),  # better than Flatten

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='sigmoid')
    ])
    return model


In [21]:
class_weights_array = compute_class_weights(train_labels)
loss_fn = weighted_binary_crossentropy(class_weights_array)

In [22]:
model = build_simple_cnn()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

In [24]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=4,
    restore_best_weights=True,
    verbose=1
)

EPOCHS = 20
model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping]
)

Epoch 1/20
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 502s 138ms/step - accuracy: 0.1771 - auc: 0.6269 - loss: 0.3198 - val_accuracy: 0.1915 - val_auc: 0.6618 - val_loss: 0.2940
Epoch 2/20
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 462s 131ms/step - accuracy: 0.2000 - auc: 0.6668 - loss: 0.2942 - val_accuracy: 0.1999 - val_auc: 0.6811 - val_loss: 0.2927
Epoch 3/20
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 457s 129ms/step - accuracy: 0.2089 - auc: 0.6756 - loss: 0.2920 - val_accuracy: 0.1587 - val_auc: 0.6625 - val_loss: 0.2987
Epoch 4/20
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 494s 127ms/step - accuracy: 0.2153 - auc: 0.6814 - loss: 0.2904 - val_accuracy: 0.2058 - val_auc: 0.6926 - val_loss: 0.2882
Epoch 5/20
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 516s 131ms/step - accuracy: 0.2212 - auc: 0.6884 - loss: 0.2887 - val_accuracy: 0.1841 - val_auc: 0.6538 - val_loss: 0.3097
Epoch 6/20
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 469s 133ms/step - accuracy: 0.2245 - auc: 0.6932 - loss: 0.2874 - val_accuracy: 0.2023 - val_auc: 0.6883 - val_loss: 0.2905
Epoc

In [25]:
model.save('multiclass.keras')

In [26]:
predictions = model.predict(test_ds)

274/274 ━━━━━━━━━━━━━━━━━━━━ 35s 117ms/step


In [34]:
for threshold in [0.1, 0.2, 0.3, 0.5]:
    pred_labels = (predictions > threshold).astype(int)
    recall = recall_score(test_labels, pred_labels, average='micro')
    precision = precision_score(test_labels, pred_labels, average='micro')
    f1 = f1_score(test_labels, pred_labels, average='micro')
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

Threshold: 0.10 | Precision: 0.1754 | Recall: 0.5977 | F1: 0.2712
Threshold: 0.20 | Precision: 0.2615 | Recall: 0.2555 | F1: 0.2585
Threshold: 0.30 | Precision: 0.5349 | Recall: 0.0394 | F1: 0.0734
Threshold: 0.50 | Precision: 0.6856 | Recall: 0.0161 | F1: 0.0316


In [28]:
pred_labels = (predictions > 0.1).astype(int) # Pick best f1 score

In [30]:
print("Classification Report:")
print(classification_report(test_labels, pred_labels, target_names=[f"Class {i}" for i in range(NUM_CLASSES)]))

precision = precision_score(test_labels, pred_labels, average='micro')
recall = recall_score(test_labels, pred_labels, average='micro')
f1 = f1_score(test_labels, pred_labels, average='micro')

print(f"Micro-Averaged Precision: {precision:.4f}")
print(f"Micro-Averaged Recall: {recall:.4f}")
print(f"Micro-Averaged F1-Score: {f1:.4f}")

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.17      0.14      0.15       530
     Class 1       0.20      0.98      0.34      1735
     Class 2       0.16      0.05      0.08       459
     Class 3       0.12      0.70      0.21       823
     Class 4       0.54      0.03      0.06       430
     Class 5       0.16      0.11      0.14       489
     Class 6       0.28      0.32      0.30       559
     Class 7       0.16      0.90      0.28      1228
     Class 8       0.11      0.50      0.18       784
     Class 9       0.21      0.97      0.34      1726
    Class 10       0.17      0.03      0.05       607
    Class 11       0.13      0.01      0.03       352

   micro avg       0.18      0.60      0.27      9722
   macro avg       0.20      0.40      0.18      9722
weighted avg       0.19      0.60      0.23      9722
 samples avg       0.19      0.60      0.27      9722

Micro-Averaged Precision: 0.1754
Micro-Averaged Recall: 

In [31]:
pred_labels = (predictions > 0.5).astype(int) # Pick best precision

In [32]:
print("Classification Report:")
print(classification_report(test_labels, pred_labels, target_names=[f"Class {i}" for i in range(NUM_CLASSES)]))

precision = precision_score(test_labels, pred_labels, average='micro')
recall = recall_score(test_labels, pred_labels, average='micro')
f1 = f1_score(test_labels, pred_labels, average='micro')

print(f"Micro-Averaged Precision: {precision:.4f}")
print(f"Micro-Averaged Recall: {recall:.4f}")
print(f"Micro-Averaged F1-Score: {f1:.4f}")

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.89      0.02      0.03       530
     Class 1       1.00      0.00      0.01      1735
     Class 2       0.00      0.00      0.00       459
     Class 3       0.54      0.01      0.02       823
     Class 4       0.88      0.02      0.03       430
     Class 5       0.00      0.00      0.00       489
     Class 6       0.72      0.16      0.26       559
     Class 7       0.59      0.03      0.06      1228
     Class 8       0.00      0.00      0.00       784
     Class 9       0.00      0.00      0.00      1726
    Class 10       0.00      0.00      0.00       607
    Class 11       0.00      0.00      0.00       352

   micro avg       0.69      0.02      0.03      9722
   macro avg       0.38      0.02      0.03      9722
weighted avg       0.43      0.02      0.03      9722
 samples avg       0.02      0.02      0.02      9722

Micro-Averaged Precision: 0.6856
Micro-Averaged Recall: 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
